In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
test = pd.read_csv('./data/test.csv')
train = pd.read_csv('./data/train.csv')

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Flatten,Dense

Using TensorFlow backend.


In [4]:
#Feature Engineering

train.loc[train['Qualification'].isnull(), 'Qualification'] = 'Special'

#test
test.loc[test['Qualification'].isnull(), 'Qualification'] = 'Special'

#Lets reduce the scale of Year_of_recruitment and Year_of_birth

train['Age'] = 2019 - train['Year_of_birth']
train['work_duration'] = 2019 - train['Year_of_recruitment']

test['Age'] = 2019 - test['Year_of_birth']
test['work_duration'] = 2019 - test['Year_of_recruitment']

#From the Employee No -- Generate Main No

train_MainNo = train.loc[:, 'EmployeeNo'].apply(lambda x: x[-5:]).astype(int)
test_MainNo = test.loc[:, 'EmployeeNo'].apply(lambda x: x[-5:]).astype(int)

In [5]:
#Label Encoding
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

def dummyEncode(df):
        columnsToEncode = list(df.select_dtypes(include=['category','object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                df[feature] = le.fit_transform(df[feature])
            except:
                print('Error encoding '+feature)
        return df
    
dummyEncode(train).head(5)

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,...,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not,Age,work_duration
0,0,1,1,0,1,2,1986,12.5,2011,1,...,41,3,0,0,0,0,0,0,33,8
1,1,2,0,1,0,2,1991,12.5,2015,0,...,52,3,1,0,0,0,0,0,28,4
2,2,1,0,1,1,2,1987,7.5,2012,0,...,42,20,1,0,0,0,0,0,32,7
3,3,1,0,1,0,3,1982,2.5,2009,0,...,42,26,1,2,0,0,1,0,37,10
4,4,4,0,1,1,3,1990,7.5,2012,0,...,77,2,1,0,0,0,1,0,29,7


In [6]:
#input the MainNo Back
train['MainNo'] = train_MainNo
test['MainNo'] = test_MainNo

In [7]:
#Lets split the Dataset

from sklearn.model_selection import train_test_split

X = train.drop(['Promoted_or_Not','EmployeeNo'], axis=1)
y = train.Promoted_or_Not

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
#To save the 'EmployeeNo column for submission'

x_tested = test.drop('EmployeeNo',axis=1) 

#And Label Encoding the Kaggle-Test set
dummyEncode(x_tested).head(5)

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Age,work_duration,MainNo
0,3,0,1,0,2,1976,7.5,2017,0,0,65,14,1,0,0,0,1,43,2,5
1,3,3,1,1,2,1991,0.0,2018,0,0,69,27,1,0,0,0,1,28,1,11
2,7,1,1,1,2,1984,7.5,2012,0,0,76,19,1,0,0,0,1,35,7,15
3,2,0,1,0,2,1984,2.5,2009,0,0,52,32,0,2,0,0,1,35,10,16
4,3,0,1,0,2,1983,7.5,2014,1,0,69,14,1,0,0,0,0,36,5,17


In [9]:
output_shape = len(set(y_train))
output_shape

2

In [10]:

model = Sequential()
model.add(Dense(100, input_dim=X.shape[1], activation='relu'))
model.add(Dense(200,activation='relu'))
model.add(Dense(150, activation='sigmoid'))
model.add(Dense(200,activation='sigmoid'))
model.add(Dense(2,activation='sigmoid'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               2100      
_________________________________________________________________
dense_2 (Dense)              (None, 200)               20200     
_________________________________________________________________
dense_3 (Dense)              (None, 150)               30150     
_________________________________________________________________
dense_4 (Dense)              (None, 200)               30200     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 402       
Total params: 83,052
Trainable params: 83,052
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(X_train,y_train,epochs=5,batch_size=100,validation_data=(X_test,y_test))

Train on 30649 samples, validate on 7663 samples
Epoch 1/5
30649/30649 [==============================] - 1s 34us/step - loss: 0.2940 - accuracy: 0.9160 - val_loss: 0.2962 - val_accuracy: 0.9130
Epoch 2/5
30649/30649 [==============================] - 1s 28us/step - loss: 0.2899 - accuracy: 0.9160 - val_loss: 0.2992 - val_accuracy: 0.9130
Epoch 3/5
30649/30649 [==============================] - 1s 28us/step - loss: 0.2899 - accuracy: 0.9160 - val_loss: 0.2957 - val_accuracy: 0.9130
Epoch 4/5
30649/30649 [==============================] - 1s 27us/step - loss: 0.2890 - accuracy: 0.9160 - val_loss: 0.2971 - val_accuracy: 0.9130
Epoch 5/5
30649/30649 [==============================] - 1s 27us/step - loss: 0.2897 - accuracy: 0.9160 - val_loss: 0.2957 - val_accuracy: 0.9130


In [13]:
#Lets do upsampling
from sklearn.utils import resample

# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
not_Promoted = X[X.Promoted_or_Not==0]
Promoted = X[X.Promoted_or_Not==1]

# upsample minority
promoted_upsampled = resample(Promoted,
                          replace=True, # sample with replacement
                          n_samples=int((len(not_Promoted))), # match number in majority class
                          random_state=42) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_Promoted,promoted_upsampled])

X_train_resampled = upsampled.drop(['Promoted_or_Not'],axis=1)
y_train_resampled = upsampled.Promoted_or_Not

In [14]:
#Lets build Xgboost and adjust the thresh-hold

import xgboost as xgb
#Learning rate = 0.05, 1000 rounds, max depth = 3-5, subsample = 0.8-1.0, colsample_bytree = 0.3-0.8, lambda = 0-5
#scale_pos_weight=10,max_delta_step=1

xgbd = xgb.XGBClassifier(learning_rate = 0.06, max_depth = 6, n_estimators= 1000,n_jobs=-1,
                         colsample_bytree = 1, reg_lambda = 5, subsample =1,
                         )

In [ ]:
xgbd.fit(X_train,y_train)

In [19]:
preds = xgbd.predict(X_test)

In [20]:
from sklearn.metrics import f1_score

#train error
predict_xgbd_train = xgbd.predict(X_train)

f1_xgbd_train = f1_score(y_train,predict_xgbd_train,average='micro')
print("TRAIN_RMSE: %f" % (f1_xgbd_train))

TRAIN_RMSE: 0.959575


In [21]:
#test error
predict_xgbd_test = xgbd.predict(X_test)

f1_xgbd_test = f1_score(y_test,predict_xgbd_test,average='micro')
print("TEST_RMSE: %f" % (f1_xgbd_test))

TEST_RMSE: 0.936448


In [22]:
preds_prob = xgbd.predict_proba(X_test)

In [23]:
details = {'MainNo':X_test['MainNo'],'Actual_Value':y_test,'Predictions':preds}
details = pd.DataFrame(details)
details['Prob_0'] = preds_prob[:,0]
details['Prob_1'] = preds_prob[:,1]

#& details['Prob_1'] 
#Adjusting Probabbilities

details.loc[:, 'New_preds'] = details.loc[:, 'Predictions']
details.loc[details.loc[:, 'Prob_1'] >= 0.30, 'New_preds'] = 1

In [24]:
f1_new = f1_score(details['New_preds'],details['Actual_Value'],average='micro')
f1_new

0.924311627300013

In [25]:
#details[details['Actual_Value'] == 0].head(15)
check = X_test.merge(details)
check[check['Actual_Value'] != check['Predictions']].head(20)

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,...,Previous_IntraDepartmental_Movement,No_of_previous_employers,Age,work_duration,MainNo,Actual_Value,Predictions,Prob_0,Prob_1,New_preds
15,3,1,1,2,2,1983,12.5,2015,1,0,...,0,0,36,4,35522,1,0,0.870882,0.129118,0
23,3,0,0,1,2,1990,5.0,2011,1,0,...,0,2,29,8,47166,1,0,0.795573,0.204427,0
59,7,0,1,1,2,1991,0.0,2018,1,0,...,0,1,28,1,54704,0,1,0.459634,0.540366,1
66,4,1,1,0,2,1990,12.5,2012,1,0,...,0,1,29,7,7973,1,0,0.890496,0.109504,0
67,8,1,0,0,2,1963,7.5,2007,1,0,...,0,1,56,12,27057,1,0,0.907998,0.092002,0
76,3,3,1,0,3,1983,12.5,2010,0,0,...,1,0,36,9,28798,1,0,0.762067,0.237933,0
111,4,1,1,0,2,1986,7.5,2011,1,0,...,0,1,33,8,33048,1,0,0.806567,0.193433,0
120,2,1,1,0,2,1979,7.5,2004,1,0,...,1,1,40,15,7975,1,0,0.711677,0.288323,0
149,1,0,1,1,2,1995,12.5,2017,1,0,...,0,1,24,2,3829,1,0,0.831812,0.168188,0
210,8,0,0,0,2,1973,7.5,2003,0,0,...,0,1,46,16,54260,1,0,0.994345,0.005655,0


In [26]:
X_test.describe().loc['mean',:].sort_values()

Past_Disciplinary_Action                   0.004045
Previous_Award                             0.021663
Previous_IntraDepartmental_Movement        0.093566
Targets_met                                0.358345
Marital_Status                             0.382096
Qualification                              0.433381
Channel_of_Recruitment                     0.475401
Gender                                     0.693071
Foreign_schooled                           0.917526
No_of_previous_employers                   1.026099
Trainings_Attended                         2.248597
Division                                   2.956414
work_duration                              5.946627
Last_performance_score                     7.747945
State_Of_Origin                           18.965549
Age                                       32.967245
Training_score_average                    55.102440
Year_of_birth                           1986.032755
Year_of_recruitment                     2013.053373
MainNo      

In [27]:
X_test.shape

(7663, 20)

In [29]:
for th in np.linspace(0.1, 0.9, 10):

    details.loc[:, 'New_preds'] = 0
    details.loc[details.loc[:, 'Prob_1'] >= th, 'New_preds'] = 1

    f1_xgbd_new_test = f1_score(y_test,details['New_preds'] ,average='micro')
    print("TEST_RMSE: %f" % (f1_xgbd_new_test))

TEST_RMSE: 0.811823
TEST_RMSE: 0.890774
TEST_RMSE: 0.920005
TEST_RMSE: 0.931097
TEST_RMSE: 0.934490
TEST_RMSE: 0.936970
TEST_RMSE: 0.937622
TEST_RMSE: 0.937100
TEST_RMSE: 0.936839
TEST_RMSE: 0.934751


In [30]:
X_train.shape

(30649, 20)

In [31]:
x_tested.shape

(16496, 20)

In [32]:
#Final Predictions
predictions = xgbd.predict(x_tested)
predictions_prob = xgbd.predict_proba(x_tested)

last = {'MainNo':x_tested['MainNo'],'predicted':predictions}
last = pd.DataFrame(last)
last['Prob_0'] = predictions_prob[:,0]
last['Prob_1'] = predictions_prob[:,1]

last.loc[:, 'New_preds'] = 0
last.loc[last.loc[:, 'Prob_1'] >= 0.9, 'New_preds'] = 1

In [ ]:
last.head(50)

In [35]:
#lets do Neural Networks

import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

In [36]:
#Lets drop year_of_birth and year of recruitment
#X_train = X_train.drop(['Year_of_birth','Year_of_recruitment'],axis=1)
X_test = X_test.drop(['Year_of_birth','Year_of_recruitment'],axis=1)

In [37]:
X_train.shape[1]
expected = len(set(y_train))

In [38]:
#Lets Normalize the dataset

num_classes = len(set(y_train))
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)

In [ ]:
models = Sequential()
models.add(Dense(48,input_dim=18,activation='sigmoid'))
models.add(Dense(96,activation='sigmoid'))
models.add(Dense(192,activation = 'softmax'))
models.add(Dense(384,activation = 'softmax'))
models.add(Dense(72,activation = 'softmax'))
models.add(Dense(24,activation='relu'))
models.add(Dense(expected,activation ='relu'))

models.compile(optimizer='Adadelta',loss='categorical_crossentropy',metrics=['accuracy'])

models.fit(X_train,y_train, epochs=5, batch_size=100,verbose=1,validation_data=(X_test,y_test))

In [ ]:
#save as Dataframe
final = {'EmployeeNo':test['EmployeeNo'],'Promoted_or_Not':predictions}
final = pd.DataFrame(final)

final.to_csv('new_submission.csv',index=False)